<h4> <i> HW01 - An Introduction to Artificial Intelligence - Dr. Fadaei 
<br>
Mohammad Montazeri - 810699269 </i> </h4>

# Curve Fitting via Genetic Algorithm
In this project, the goal is to write a program that uses genetic algorithm to find the best fitting polynomial function that (almost) passes the points given as th input of the problem. 

### Abstract 
In this code, the user gives some points and specifies some initial settings to get the code running. The code, will eventually return a polynomial with integer coefficients that can represent one of the best lines that (mostly) crosses the given points. The mentioned initial settings are:
1. the order (degree) of the expected polynomial as the result
2. lower and upper limits of the integer coefficients in the expected polynomial 
3. how many initial guesses must the algorithm start with
4. the probability of applying *crossover* on a set of chromosomes
5. the probability of applying *mutation* on a set of chromosomes
6. how many generations should the code create on the most  
This code is supposed to work for polynomials ordered 1 or more. The higher the order (degree) of polynomials, the longer it takes the code to return the result. For high orders, the number of initial guesses and the maximum number of generations must increase, so that the code can gradually converge to a better answer. 

### Inputs:
There are some test-cases at the end of the code that represent the way one can specify the following inputs for one's desired problem.
- Some points of the diagram
- Order of the polynomial 
- Domain of Coefficients 
- Number of initial population
- Rate of crossover and mutation
- Number of iterations

### Algorithm of the Code
In this project, to utilize genetic algorithm, the chromosome is defined as a list of coefficients of a polynomial which is supposed to be the answer of the problem. Every chromosome is made of some genes, which are the mentioned coefficients. Since these coefficients are supposed to be integer, every gene can be assumed an `int` variable with some low and high limits the client gives as the problem parameter. 
In this code, a **class** is defined as the main part of the program. There are a bunch of methods in this class that handle all of the operations of the algorithm. In this section, we are to discuss every function a little closer on the bottom of each code snippet. 

In [ ]:
from random import *
from time import time


class Genetic:
    def __init__(self, n, lowLim, highLim, numPopulation, crossoverRate, mutationRate, numIteration, *points) -> None:
        self.order = n
        self.domain = [lowLim, highLim]
        self.N_population = numPopulation
        self.P_cross = crossoverRate
        self.P_mut = mutationRate
        self.N_iteration = numIteration
        self.points = list(points)
        self.initial_population()

#### \_\_init\_\_
This function initializes any object of this class by taking the mentioned inputs and assigning them to class variables. Then it calls the next method to create the initial guesses of the problem.

In [ ]:
    def initial_population(self):
        self.population = []
        for i in range(self.N_population):
            Chromosome = list(randint(*self.domain)
                              for j in range(self.order + 1))
            while Chromosome[0] == 0:
                Chromosome[0] = randint(*self.domain)

            self.population.append([Chromosome, 0])


#### initial_population
This function uses a loop, iterated as many times as the number the client had specified as the size of initial guesses list. Within the loop, every *chromosome* is created with random *genes* using `random` package. The number of genes in each chromosome must be one more than the order of the expected polynomial, because, for instance, a second order polynomial, has 3 coefficients as:
`ax^2 + bx + c`.
Also, with another `while` loop, we control that the first gene of each chromosome is not **zero**; otherwise, the order of the result might be less than the specified value.
All created chromosomes are saved alongside their initial fitness in a list and then added to the *population* list which is going to be the platform of our upcoming functions.

In [ ]:
    def crossover(self):
        self.crossed = []
        for i in range(1, self.N_population, 2):
            parents = [self.population[i-1], self.population[i]]
            if random() <= self.P_cross:
                slicePoint = randint(0, self.order - 1)
                child1 = parents[0][0][:slicePoint] + \
                    parents[1][0][slicePoint:]
                child2 = parents[1][0][:slicePoint] + \
                    parents[0][0][slicePoint:]
                self.crossed.extend([[child1, 0], [child2, 0]])

            else:
                self.crossed.extend(parents)

        if self.N_population % 2:
            self.crossed.append(self.population[-1])

        self.population.clear()
        self.population = self.crossed.copy()
        self.fitness()
        self.sorter()


#### crossover
Here, a loop iterates on all members of the *population* list with a step of 2. Like this, all sample chromosomes are selected as the parents, two by two. From this parents will the next generation (two children from every pair of parents) be born. To see if crossover process must be applied on each chromosome pair or not, a random number (between 0 & 1) is generated. If it was less than the specified crossover probability, the process will be done; otherwise, those specific parents will be added directly to the next generation. To do the crossover process, a random index of the chromosomes must be selected as `slicepoint`. Note that it can not be the **last** index; that's why the limits of `randint` are chosen like this. 
If the number of members in *population* list was ODD, the last member won't have a pair to become a parent, so it is directly added to the next generation named `crossed`. 
The main generation, then, will be replaced with this function's result (*crossed* list). Afterwards, the fitness of each chromosome will be measured, and according to that, the population will be sorted. These are done by the upcoming methods `fitness` & `sorter`.

In [ ]:
    def mutation(self):
        for i in range(self.N_population):
            if random() <= self.P_mut:
                slicePoint = randint(0, self.order)
                substitution = randint(*self.domain)
                while slicePoint == 0 and substitution == 0:
                    substitution = randint(*self.domain)
                self.population[i][0][slicePoint] = substitution

        self.fitness()
        self.sorter()


#### mutation
Here, looping again over all chromosomes, with the same route as that used in *crossover*, we decide to do the mutation on a chromosome or not. Once more, we define *slicepoint* as the index which must be mutated in the chromosome. This time, even the last index can be mutated as well. A random integer number is used as the substitution of the previous gene. Once again, we use a loop to decline the changes if the code tries to assign zero to the first gene (coefficient of the largest order in the polynomial).
Like crossover, the fitness of the chromosomes is measured and they are sorted based on that.

In [ ]:
    def fitness(self):
        for i in range(self.N_population):
            mean_err = 0
            for pt in self.points:
                x = pt[0]
                y_estimation = 0
                for j in range(self.order + 1):
                    power = self.order - j
                    coeff = self.population[i][0][j]
                    y_estimation += coeff * x ** power
                error = abs(y_estimation - pt[1])
                mean_err += error
            mean_err /= len(self.points)
            mean_err = mean_err**(1/3)
            fit = 1/(1+mean_err)
            self.population[i][1] = fit


#### fitness
The idea is to find the difference between the actual value of y and the estimation of y which one chromosome suggests. Looping over all chromosomes, the mentioned difference (named `error`) is calculated for each point the client has given to the program. All these *error*s calculated for a chromosome will be added to the `mean_err` of that chromosome. Then its 3rd root will be calculated so that the error value might be shrunk for huge actual-and-estimated-y-differences.
We want a fitness factor which is between **zero** and **one**, while the closer the factor to 1, the better the chromosome. To implement this idea, the fitness is supposed to be:
$$ 1 / (1+mean_err) $$
This value is assigned to the 2nd index of each chromosome-**list** which, as mentioned before, had been set to zero as the initial fitness of each chromosome.

In [ ]:
    def sorter(self):
        self.population = sorted(
            self.population, key=lambda x: x[1], reverse=True)


#### sorter
using Python's `sorted` method, we can easily sort the *population*, 
[based on the fitness of each chromosome]('#', 'important'). To do so, a `lambda` syntax is used as can be seen. Also, to sort the list **descendingly**, the *reverse* key is set *True*.

In [ ]:
    def answer(self):
        ans, power = '   y = ', self.order
        for gene in self.population[0][0]:
            if power:
                ans += f'{gene}*x^{power} + '
            else:
                ans += f'{gene}   '
            power -= 1
        return ans


#### answer
This is just a simple mathematical method that gets the first chromosome of the sorted population list as the answer of the problem. That's because the highest fitness (max = 1) will be sitting in the beginning of the population list. Then, using this chromosome, the string representations of coefficients (genes) within a polynomial function is created and returned as the final answer.

In [ ]:
    def main(self):
        for i in range(self.N_iteration):
            # shuffle(self.population)
            self.crossover()
            self.mutation()

            if self.population[0][1] == 1:
                print(
                    f'We\'ve reached the exact answer as: {self.answer()} in #{i}th iteration.')
                return 1
        print(
            f'We couldn\'t find the perfect answer; but here\'s the best answer we could get: {self.answer()}')
        return 0


#### main
This is the method that should be called by the client to start all of the methods mentioned earlier. Here, as may times as the user has specified as the maximum number of iterations, the main loop of the algorithm will run. Within the loop, the crossover method acts on the population and mutation after that, on a row. Hereby, the next generation is created, sorted and saved in the main *population* list. Then, the first chromosome, which is the one that might be the answer (it has the largest fitness in the list) is checked. If its fitness was exactly 1, it would be the exact answer; else, the loop goes on and on, until it reaches the answer on one of its generations, or the loop-time (# of iterations) ends.
After that, the answer will be printed using `answer()` function. This answer might be the exact solution of the problem, or the closest one to it that the code could obtain with the specified parameters given to it as its inputs.

In [ ]:
# Running the code:
start = time()

# test case 1 ----------------------------------------------------------------------------------------
# (n=1, lowLim=0, highLim=9, numPopulation=100, crossoverRate=0.1, mutationRate=0.1, numIteration=100)
Points = [(1, 2), (2, 4)]
sample = Genetic(1, 0, 9, 100, 0.1, 0.1, 100, *Points)
sample.main()

# test case 2 -------------------------------------------------------------------------------------------
# (n=2, lowLim=-10, highLim=35, numPopulation=1000, crossoverRate=0.5, mutationRate=0.5, numIteration=1000)
# Points = [(0, 10), (-1, 0), (-0.5, 0)]
# sample = Genetic(2, -10, 35, 1000, 0.5, 0.5, 1000, *Points)
# sample.main()

# test case 3 -------------------------------------------------------------------------------------------
# (n=3, lowLim=-9, highLim=9, numPopulation=1000, crossoverRate=0.5, mutationRate=0.5, numIteration=1000)
# Points = [(0, 1), (1, 0), (2, -5), (-1, -8)]
# sample = Genetic(3, -9, 9, 1000, 0.5, 0.5, 1000, *Points)
# sample.main()

# test case 4 ----------------------------------------------------------------------------
# (n=4, lowLim=0, highLim=20, numPopulation=1000, crossoverRate=0.9, mutationRate=0.8, numIteration=1000)
# Points = [(0, 1), (1, 43), (3, 1477), (5, 10231), (7, 37969)]
# sample = Genetic(4, 0, 20, 1000, 0.9, 0.8, 1000, *Points)
# sample.main()
# true answer: y = 15x^{4} +3x^{3} +18x^{2} +6x +1

# test case 5 ----------------------------------------------------------------------------
# (n=5, lowLim=-10, highLim=10, numPopulation=1000, crossoverRate=1, mutationRate=0.9, numIteration=10000)
# Points = [(0, 1), (1, 2), (-1, 12), (-2, 149), (2, -171), (-3, 1054)]
# sample = Genetic(5, -9, 9, 1000, 1, 0.9, 1000, *Points)
# sample.main()
# true answer: y = -5x^{5} -3x^{4} +9x^{2} +1

# test case 6 ----------------------------------------------------------------------------
# (n=5, lowLim=-10, highLim=10, numPopulation=1000, crossoverRate=1, mutationRate=0.9, numIteration=10000)
# Points = [(0, 1), (1, 4), (-1, 10), (-2, 139), (2, -161), (-3, 1024)]
# sample = Genetic(5, -10, 10, 1000, 1, 0.9, 10000, *Points)
# sample.main()
# true answer: y = -5x^{5} -3x^{4} +x^{3} +9x^{2} +x +1

# test case 7 ----------------------------------------------------------------------------
# (n=6, lowLim=0, highLim=5, numPopulation=1000, crossoverRate=1, mutationRate=0.9, numIteration=1000)
# Points = [(0, 0), (-1, -1), (1, 3), (-2, 48), (2, 80), (-3, 675), (3, 783)]
# sample = Genetic(6, 0, 5, 1000, 1, 0.9, 1000, *Points)
# sample.main()
# true answer: y = x^{6} +2x^{3}

# test case 8 ----------------------------------------------------------------------------
# (n=8, lowLim=-25, highLim=-10, numPopulation=1000, crossoverRate=1, mutationRate=0.9, numIteration=1000)
# Points = [(0, 0), (-1, -22), (1, -22), (-2, -5632), (2, -5632), (-3, -144342), (3, -144342), (-4, -1441792), (4, -1441792)]
# sample = Genetic(8, -23, 2, 1000, 1, 1, 1000, *Points)
# sample.main()
# true answer: y = -22x^{8}

stop = time()
print(f'runtime = {stop - start}')


#### ending
Finally, some test cases with different polynomial orders are provided. The time that the code runs until giving the result is also measured by *time* package and shown in the end.

### Questions
1. > Enormous number of initial population causes more time and resources consumption in each generation and thus, more delay until the answer is returned. On the other hand, very small number of initial population reduced the variety of answers and might deprive our algorithm to reach the answer on time.
2. > This phenomenon might also result in accelerating time and resources saturation, because it increases time consumption in every generation. It might also cause creating more deviated and useless chromosomes in some late generations.
3. > We shouldn't eliminate any of the two methods *crossover* & *mutation*. That's because crossover uses 2 of the better answers we've had up to now, and merges them. So the changes in crossover are more significant than in mutation. But without mutation, a constant number of genes repeat in every generation and the algorithm will lack the diversity it needs. So, the combination of crossover and mutation makes the code faster and more accurate.
4. > The number of initial generation, number of iterations, crossover and mutation rate and the limits of the coefficients (genes) are very effective in the final answer of the code, and its time consumption. So, a closer choice of these parameters to the reality of the answer might help the algorithm.
5. > If the number of initial population was too small, or the limitations of genes were too restricted, the crossover and mutation might loose their proper functionality and cause a repeated chromosome set and constant overall fitness in late generations. To prevent these phenomenon, the mentioned parameters must be chosen wisely.
6. > We have used a finite number as the maximum number of iterations, so that the code will never get stuck in an infinite loop. Besides, we can decrease the goal fitness (currently 1) of the algorithm gradually, so if the code running takes too long, it lowers the level of expectation of the algorithm, and ends the loop.
7. > The time and resources consumption of the algorithm increases significantly with increasing the order of the polynomial. In that case, the code might not reach the exact answer except with very large numbers of initial population and iterations. Also, the randomness of the methods affect the performance; so, if the code didn't reach the precise answer of the problem with one run, more tries must be employed so that it might reach a more proper result in other try-outs.
8. > Decreasing the number of points might prevent the problem from having a single unique answer. So, the code might return different answers each time it is run, and they might be all correct. Increasing th number of points might result in more time consumption, because in `fitness` method, more points must be checked and evaluated for every chromosome and this may become a devastating process.


<h4>by: <br> Mohammad Montazeri
<br>
810699269 </h4>
